# Import Modules

In [38]:
import os
import random

# Create Classes

In [39]:
class FullTrackList():
    def __init__(self, filename):
        self.filename(filename)
        
    def filename(self, filename):
        """Setter used to set the filename and validate it, also gets tracks from the file and puts them into a list"""

        # Tests if the CSV file supplied is valid and exists
        if type(filename) == str and filename.endswith(".csv") and os.path.isfile(filename):
            self._filename = filename
            self._tracks = []
            with open(self._filename, "r") as fh:
                for line in fh.readlines():
                    split_line = line.split(",")
                    self._tracks.append(Track(id=split_line[0], artist=split_line[1], title=split_line[2], time=TimeMS(split_line[3])))
            
        else:
            raise TypeError("Error, invalid file selected")
        
    def get_filename(self):
        """Getter, used to get the filename"""
        return self._filename

    def get_tracks(self):
        """Getter, used to get the list of tracks in the tracklist"""
        return self._tracks


class Playlist():
    def __init__(self, track_ids, tracklist):
        self._track_ids = track_ids
        self.tracks = []
        self._tracklist = tracklist
        self._time = 0
        for track in self._tracklist.get_tracks():
            if track.id in self._track_ids:
                self.tracks.append(track)
                self._time += int(track.time.in_seconds())

    def show_playlist(self):
        # Create a Time object using self._time, which is the total time in seconds for each track
        # Then, represent this Time object in minutes to get the total track time in minutes
        print("Playing Time:", TimeMS(str(self._time)).in_minutes())
        for track in self.tracks:
            print("\t" + str(track.id) + "\t" + str(track.artist) + ":", track.title)

    def add_track(self, track_id):
        for track in self._tracklist.get_tracks():
            if track.id == track_id:
                self.tracks.append(track)
                self._time += int(track.time.in_seconds())

    def remove_track(self, track_id):
        for track in self.tracks:
            if track.id == track_id:
                self.tracks.remove(track)
                self._time -= int(track.time.in_seconds())

    def shuffle_playlist(self):
        # using Fisher–Yates shuffle Algorithm to shuffle a list
        for i in range(len(self.tracks)-1, 0, -1):
            
            # Pick a random index from 0 to i
            j = random.randint(0, i)
        
            # Swap arr[i] with the element at random index
            self.tracks[i], self.tracks[j] = self.tracks[j], self.tracks[i]


    def sort_playlist(self):
        self.tracks.sort(key=lambda x: (x.artist, x.title))


class Track():
    def __init__(self, id, artist, title, time):
        self.id = id
        self.artist = artist
        self.title = title
        self.time = time


class TimeMS():
    def __init__(self, time):
        if ":" in time:
            self.min_to_sec(time)
        
        else:
            self.sec_to_min(time)

    def min_to_sec(self, time):
        self._full_mins = time
        self._full_mins_split = time.split(":")
        self._min_split, self._sec_split = self._full_mins_split[0], self._full_mins_split[1]
        self._full_seconds = (int(self._min_split) * 60) + int(self._sec_split)

    def sec_to_min(self, time):
        self._full_seconds = time
        self._sec_split = int(self._full_seconds)
        self._min_split = 0
        while self._sec_split > 60:
            self._sec_split -= 60
            self._min_split += 1

        if len(str(self._sec_split)) == 1:
            self._sec_split = "0" + str(self._sec_split)
            
        else:
            self._sec_split = str(self._sec_split)

        self._full_mins = str(self._min_split) + ":" + self._sec_split

    def in_seconds(self):
        return self._full_seconds

    def in_minutes(self):
        return self._full_mins



# Testing

In [40]:
tlist = FullTrackList("music.csv")
print(tlist.get_filename())

music.csv


In [41]:
pl = Playlist(["1", "2", "3"], tlist)
pl.show_playlist()

Playing Time: 11:50
	1	Taylor Swift: Everything Has Changed
	2	Mumford and Sons: Little Lion Man
	3	Hozier: Sedated


In [42]:
pl.add_track("7")
pl.add_track("8")
pl.add_track("12")
pl.show_playlist()

Playing Time: 22:05
	1	Taylor Swift: Everything Has Changed
	2	Mumford and Sons: Little Lion Man
	3	Hozier: Sedated
	7	Hozier: Jackie and Wilson
	8	Hozier: Take Me To Church
	12	Miley Cyrus: Jolene


In [43]:
pl.add_track("13")
pl.remove_track("1")
pl.shuffle_playlist()
pl.show_playlist()

Playing Time: 21:37
	8	Hozier: Take Me To Church
	12	Miley Cyrus: Jolene
	7	Hozier: Jackie and Wilson
	13	Florence + The Machine: Dog Days are Over
	2	Mumford and Sons: Little Lion Man
	3	Hozier: Sedated


In [44]:
pl.sort_playlist()
pl.show_playlist()

Playing Time: 21:37
	13	Florence + The Machine: Dog Days are Over
	7	Hozier: Jackie and Wilson
	3	Hozier: Sedated
	8	Hozier: Take Me To Church
	12	Miley Cyrus: Jolene
	2	Mumford and Sons: Little Lion Man


In [45]:
pl

Playing Time: 21:37
	13	Florence + The Machine: Dog Days are Over
	7	Hozier: Jackie and Wilson
	3	Hozier: Sedated
	8	Hozier: Take Me To Church
	12	Miley Cyrus: Jolene
	2	Mumford and Sons: Little Lion Man


TypeError: __repr__ returned non-string (type NoneType)